<a href="https://colab.research.google.com/github/adenurchalisa/DeepLearning_finalproject/blob/main/mobilenetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/DL/dataset/dataset_augmented"
IMG_SIZE = 224
BATCH_SIZE = 16
SEED = 42

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)
val_gen = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)
NUM_CLASSES = train_gen.num_classes

Found 757 images belonging to 3 classes.
Found 189 images belonging to 3 classes.


In [ ]:
def build_model(hp):
    base_model = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = hp.Boolean("fine_tune", default=False)
    model = keras.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(
        units=hp.Int("dense_units1", min_value=64, max_value=512, step=64),
        activation="relu"
    ))
    model.add(layers.Dropout(rate=hp.Float("dropout1", min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(
        units=hp.Int("dense_units2", min_value=32, max_value=256, step=32),
        activation="relu"
    ))
    model.add(layers.Dropout(rate=hp.Float("dropout2", min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-5, 1e-3, sampling="log")
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    directory="kt_dir",
    project_name="mobilenetv2_cleanliness"
)

callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=2),
]

tuner.search(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=callbacks
)

Trial 10 Complete [00h 03m 20s]
val_accuracy: 0.8042327761650085

Best val_accuracy So Far: 0.9047619104385376
Total elapsed time: 00h 48m 27s


In [ ]:
best_model = tuner.get_best_models(1)[0]
best_model.save("/content/mobilenetv2_best_tuned.h5")

val_loss, val_acc = best_model.evaluate(val_gen)
print(f"Best validation accuracy: {val_acc:.3f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 223ms/step - accuracy: 0.9316 - loss: 0.2394
Best validation accuracy: 0.905


In [ ]:
tuner.results_summary()

Results summary
Results in kt_dir/mobilenetv2_cleanliness
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
fine_tune: False
dense_units1: 384
dropout1: 0.2
dense_units2: 256
dropout2: 0.30000000000000004
learning_rate: 0.0005986673858497197
Score: 0.9047619104385376

Trial 00 summary
Hyperparameters:
fine_tune: False
dense_units1: 192
dropout1: 0.2
dense_units2: 128
dropout2: 0.30000000000000004
learning_rate: 0.00026051698784545477
Score: 0.8730158805847168

Trial 07 summary
Hyperparameters:
fine_tune: False
dense_units1: 384
dropout1: 0.2
dense_units2: 256
dropout2: 0.30000000000000004
learning_rate: 6.72349583281083e-05
Score: 0.8359788656234741

Trial 09 summary
Hyperparameters:
fine_tune: True
dense_units1: 320
dropout1: 0.2
dense_units2: 192
dropout2: 0.4
learning_rate: 9.196308750140528e-05
Score: 0.8042327761650085

Trial 03 summary
Hyperparameters:
fine_tune: False
dense_units1: 384
dropout1: 0.30000000000000004
dense_un